# Solving a complex task with a multi-agent hierarchy

This notebook is part of the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course), a free Course from beginner to expert, where you learn to build Agents.

![Agents course share](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/communication/share.png)

The reception is approaching! With your help, Alfred is now nearly finished with the preparations.

But now there's a problem: the Batmobile has disappeared. Alfred needs to find a replacement, and find it quickly.

Fortunately, a few biopics have been done on Bruce Wayne's life, so maybe Alfred could get a car left behind on one of the movie set, and re-engineer it up to modern standards, which certainly would include a full self-driving option.

But this could be anywhere in the filming locations around the world - which could be numerous.

So Alfred wants your help. Could you build an agent able to solve this task?

> 👉 Find all Batman filming locations in the world, calculate the time to transfer via a cargo plane to there, and represent them on a map, with a color varying by a cargo plane transfer time. Also represent some supercar factories with the same cargo plane transfer time.

Let's build this!

In [ ]:
import sys, subprocess
print("Interpreter:", sys.executable)
subprocess.run([sys.executable, "-m", "pip", "install", "-U", "kaleido"], check=False)
subprocess.run([sys.executable, "-m", "pip", "show", "kaleido"], check=False)


In [ ]:
#pip install smolagents[litellm] plotly geopandas shapely kaleido -q

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# We first make a tool to get the cargo plane transfer time.
import math
from typing import Optional, Tuple

from smolagents import tool


@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750.0,  # Average speed for cargo planes
) -> float:
    """
    Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750 km/h for typical cargo planes)

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)

    # Extract coordinates
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371.0

    # Calculate great-circle distance using the haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # Add 10% to account for non-direct routes and air traffic controls
    actual_distance = distance * 1.1

    # Calculate flight time
    # Add 1 hour for takeoff and landing procedures
    flight_time = (actual_distance / cruising_speed_kmh) + 1.0

    # Format the results
    return round(flight_time, 2)


print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))

For the model provider, we use Together AI, one of the new [inference providers on the Hub](https://huggingface.co/blog/inference-providers)!

Regarding the GoogleSearchTool: this requires either having setup env variable `SERPAPI_API_KEY` and passing `provider="serpapi"` or having `SERPER_API_KEY` and passing `provider=serper`.

If you don't have any Serp API provider setup, you can use `DuckDuckGoSearchTool` but beware that it has a rate limit.

In [ ]:
import os
from PIL import Image
from smolagents import CodeAgent, GoogleSearchTool, InferenceClientModel, VisitWebpageTool


model = InferenceClientModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")

We can start with creating a baseline, simple agent to give us a simple report.

In [ ]:
task = """Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.
Also give me some supercar factories with the same cargo plane transfer time."""

In [ ]:
import os
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")
print(os.environ["SERPAPI_API_KEY"])

In [ ]:
agent = CodeAgent(
    model=model,
    tools=[GoogleSearchTool(), VisitWebpageTool(), calculate_cargo_travel_time],
    additional_authorized_imports=["pandas"],
    max_steps=20,
)

In [ ]:
result = agent.run(task)

In [ ]:
result

We could already improve this a bit by throwing in some dedicated planning steps, and adding more prompting.

In [ ]:
agent.planning_interval = 4

detailed_report = agent.run(f"""
You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

{task}
""")

print(detailed_report)

In [ ]:
detailed_report

Thanks to these quick changes, we obtained a much more concise report by simply providing our agent a detailed prompt, and giving it planning capabilities!

💸 But as you can see, the context window is quickly filling up. So **if we ask our agent to combine the results of detailed search with another, it will be slower and quickly ramp up tokens and costs**.

➡️ We need to improve the structure of our system.

## ✌️ Splitting the task between two agents

Multi-agent structures allow to separate memories between different sub-tasks, with two great benefits:
- Each agent is more focused on its core task, thus more performant
- Separating memories reduces the count of input tokens at each step, thus reducing latency and cost.

Let's create a team with a dedicated web search agent, managed by another agent.

The manager agent should have plotting capabilities to redact its final report: so let us give it access to additional imports, including `plotly`, and `geopandas` + `shapely` for spatial plotting.

In [ ]:
import os
from PIL import Image
from smolagents import CodeAgent, GoogleSearchTool, InferenceClientModel, VisitWebpageTool

model = InferenceClientModel(
    "Qwen/Qwen2.5-Coder-32B-Instruct", provider="together", max_tokens=8096
)

import os
from dotenv import load_dotenv


# Simple approach - looks for .env in current working directory
load_dotenv()

key = os.getenv("SERPAPI_API_KEY")
print("Server Key:", key)

os.environ["SERPER_API_KEY"] = key

web_agent = CodeAgent(
    model=model,
    tools=[
        GoogleSearchTool(provider="serper"),
        VisitWebpageTool(),
        calculate_cargo_travel_time,
    ],
    name="web_agent",
    description="Browses the web to find information",
    verbosity_level=0,
    max_steps=10,
)

The manager agent will need to do some mental heavy lifting.

So we give it the stronger model [DeepSeek-R1](https://huggingface.co/deepseek-ai/DeepSeek-R1), and add a `planning_interval` to the mix.

In [ ]:
import os

key = os.getenv("OPENAI_API_KEY")
print("Open API Key:",key)
os.environ["OPENAI_API_KEY"] = key

In [11]:
from smolagents.utils import encode_image_base64, make_image_url
from smolagents import OpenAIServerModel


def check_reasoning_and_plot(final_answer, agent_memory):
    multimodal_model = OpenAIServerModel("gpt-4o", max_tokens=8096)
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(
    model=InferenceClientModel("gpt-4o-mini", provider="openai", api_key=os.environ["OPENAI_API_KEY"], max_tokens=4096),
    tools=[calculate_cargo_travel_time],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "os",
        "json",
        "math",
        "random",
        "collections",
        "datetime",
        "statistics",
        "queue",
        "re",
        "numpy",
        "pandas",
        "geopandas",
        "shapely",
        "plotly",                 # base
        "plotly.io",              # for write_html / renderers
        "plotly.express",         # px
        "plotly.express.colors",  # fixes the px.colors access error
        "plotly.graph_objects",   # use go instead of graph_objs
        "plotly.figure_factory",  # if the agent tries ff.*
         "_plotly_utils.colors",          # <- private utils (agent is reaching here)
        "_plotly_utils.colors.sequential",   # <- specifically needed for px.colors.sequential.Magma
         "plotly.express.colors",           # optional, if agent uses px.colors
        "_plotly_utils.colors","_plotly_utils.colors.sequential",  # if agent reaches these        
        "kaleido","kaleido.scopes","kaleido.scopes.plotly",
        "unicodedata",
        "subprocess",
        "stat",
        "ntpath",
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,    
)

# manager_agent.run("""
# Write Python code that does exactly:

# import pandas as pd
# import plotly.express as px
# import plotly.io as pio

# df = pd.DataFrame({"x":[0,1,2], "y":[0,1,0]})
# fig = px.line(df, x="x", y="y", title="kaleido smoke test")

# # Try PNG first (needs kaleido)
# try:
#     fig.write_image("kaleido_smoke.png")
#     png_status = "PNG_OK"
# except Exception as e:
#     print("PNG export failed:", e)
#     png_status = "PNG_FAIL"

# # Always save HTML as fallback (no kaleido needed)
# pio.write_html(fig, "kaleido_smoke.html", auto_open=False)

# final_answer({"png": png_status, "html": "HTML_OK"})
# """)

Let us inspect what this team looks like:

In [12]:
manager_agent.visualize()

CodeAgent | gpt-4o-mini
├── ✅ Authorized imports: ['os', 'json', 'math', 'random', 'collections', 'datetime', 'statistics', 'queue', 're',
│   'numpy', 'pandas', 'geopandas', 'shapely', 'plotly', 'plotly.io', 'plotly.express', 'plotly.express.colors', 
│   'plotly.graph_objects', 'plotly.figure_factory', '_plotly_utils.colors', '_plotly_utils.colors.sequential', 
│   'plotly.express.colors', '_plotly_utils.colors', '_plotly_utils.colors.sequential', 'kaleido', 
│   'kaleido.scopes', 'kaleido.scopes.plotly', 'unicodedata', 'subprocess', 'stat', 'ntpath']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name                        ┃ Description                           ┃ Arguments                             ┃
│   ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ calculate_cargo_travel_time │ Calculate the travel time for a cargo │ origin_coords (`array`): Tuple of     │
│   │                             │ plane between two points on Earth     │ (latitude, longitude) for the         │
│   │                             │ using great-circle distance.          │ starting point                        │
│   │                             │                                       │ destination_coords (`array`): Tuple   │
│   │                             │                                       │ of (latitude, longitude) for the      │
│   │                             │                                       │ destination                           │
│   │                             │                                       │ cruising_speed_kmh (`number`):        │
│   │                             │                                       │ Optional cruising speed in km/h       │
│   │                             │                                       │ (defaults to 750 km/h for typical     │
│   │                             │                                       │ cargo planes)                         │
│   │ final_answer                │ Provides a final answer to the given  │ answer (`any`): The final answer to   │
│   │                             │ problem.                              │ the problem                           │
│   └─────────────────────────────┴───────────────────────────────────────┴───────────────────────────────────────┘
└── 🤖 Managed agents:
    └── web_agent | CodeAgent | Qwen/Qwen2.5-Coder-32B-Instruct
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Browses the web to find information
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name                        ┃ Description                       ┃ Arguments                         ┃
            ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search                  │ Performs a google web search for  │ query (`string`): The search      │
            │                             │ your query then returns a string  │ query to perform.                 │
            │                             │ of the top search results.        │ filter_year (`integer`):          │
            │                             │                                   │ Optionally restrict results to a  │
            │                             │                                   │ certain year                      │
            │ visit_webpage               │ Visits a webpage at the given url │ url (`string`): The url of the    │
            │                             │ and reads its content as a        │ webpage to visit.                 │
            │                             │ markdown string. Use this to      │                                   │
            │                             │ browse webpages.                  │       

In [13]:
SAFETY_PREFACE = """
PLOTTING & EXPORT RULES (sandbox-safe):

1) Imports you MAY use for plotting:
   - import plotly.express as px
   - import plotly.graph_objects as go   # optional
   - import plotly.io as pio
   - import pandas as pd, numpy as np, json, os

2) DO NOT:
   - Do NOT call fig.show()  (it fails in this sandbox).
   - Do NOT import px.colors or private modules like _plotly_utils.*.
   - Do NOT import plotly.graph_objs (use plotly.graph_objects as go instead).

3) Colorscale:
   - If you need a colorscale, pass the NAME string (e.g., color_continuous_scale="Magma" or "Plasma")
     instead of referencing px.colors.*.

4) Figure contract:
   - Always create a Plotly Figure in a variable named: fig
   - Ensure it is a valid figure (not None) before exporting.

5) Export policy (PNG + HTML):
   - First try to write a PNG (requires kaleido):
       try:
           fig.write_image("saved_map.png")
           png_status = "PNG_OK"
       except Exception as e:
           print("PNG export failed:", e)
           png_status = "PNG_FAIL"
   - Always write an HTML backup (no kaleido needed):
       pio.write_html(fig, "saved_map.html", auto_open=False)

6) Final answer:
   - Return the figure: final_answer(fig)
   - Do NOT attempt inline rendering.

7) DataFrame columns vary. Before plotting, inspect df columns and adapt parameter names:
   - Example for scatter map:
       fig = px.scatter_map(
           df,
           lat="<your-lat-col>",
           lon="<your-lon-col>",
           text="<your-label-col>",
           color="<your-color-col>",
           color_continuous_scale="Magma",
           size_max=15,
           zoom=1,
           title="<your-title>"
       )
"""


manager_agent.run(SAFETY_PREFACE + """
Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W).
Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in total.
Represent this as spatial map of the world, with the locations represented as scatter points with a color that depends on the travel time, and save it to saved_map.png!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
     color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)

Never try to process strings using code: when you have a string to read, just print it and you'll see it.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ PLOTTING & EXPORT RULES (sandbox-safe):                                                                         │
│                                                                                                                 │
│ 1) Imports you MAY use for plotting:                                                                            │
│    - import plotly.express as px                                                                                │
│    - import plotly.graph_objects as go   # optional                                                             │
│    - import plotly.io as pio                                                                                    │
│    - import pandas as pd, numpy as np, json, os                                                                 │
│                                                                                                                 │
│ 2) DO NOT:                                                                                                      │
│    - Do NOT call fig.show()  (it fails in this sandbox).                                                        │
│    - Do NOT import px.colors or private modules like _plotly_utils.*.                                           │
│    - Do NOT import plotly.graph_objs (use plotly.graph_objects as go instead).                                  │
│                                                                                                                 │
│ 3) Colorscale:                                                                                                  │
│    - If you need a colorscale, pass the NAME string (e.g., color_continuous_scale="Magma" or "Plasma")          │
│      instead of referencing px.colors.*.                                                                        │
│                                                                                                                 │
│ 4) Figure contract:                                                                                             │
│    - Always create a Plotly Figure in a variable named: fig                                                     │
│    - Ensure it is a valid figure (not None) before exporting.                                                   │
│                                                                                                                 │
│ 5) Export policy (PNG + HTML):                                                                                  │
│    - First try to write a PNG (requires kaleido):                                                               │
│        try:                                                                                                     │
│            fig.write_image("saved_map.png")                                                                     │
│            png_status = "PNG_OK"                                                                                │
│        except Exception as e:                                                                                   │
│            print("PNG export failed:", e)                                                                       │
│            png_status = "PNG_FAIL"                                                                              │
│    - Always write an HTML backup (no kaleido needed):                                                           │
│        pio.write_html(fig, "saved_map.html", auto_open=False)                                                   │
│                                                                                                                 │
│ 6) Final answer:                                      

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
## 1. Facts survey

### 1.1. Facts given in the task
- Current location: Gotham City at coordinates (40.7128° N, 74.0060° W).
- We need to find Batman filming locations around the world.
- We also need to identify supercar factories.
- We need to calculate the transfer time via cargo plane to these locations from Gotham City.
- The final output should be a spatial map with locations represented as scatter points colored by travel time, and
the map should be exported as a PNG and HTML file.

### 1.2. Facts to look up
- List of Batman filming locations globally, including their coordinates (latitude and longitude).
- List of supercar factories globally, including their coordinates (latitude and longitude).
- The cruising speed of cargo planes, which may typically be around 750 km/h, but should be confirmed.
- Any specific details necessary on how to create a spatial map using Plotly to ensure proper implementation of the
plotting requirements.

### 1.3. Facts to derive
- Calculate great-circle distances and corresponding travel times from Gotham City to each of the identified 
filming locations and supercar factories.
- Compile a single DataFrame containing coordinates, names, and calculated travel times for plotting.
- Assign a color coding based on travel times for visualization on the map.

## 2. Plan
1. Search for and gather a list of Batman filming locations worldwide, compiling their coordinates into an 
appropriate format.
2. Search for and gather a list of supercar factories worldwide, compiling their coordinates into an appropriate 
format.
3. Confirm the cruising speed of cargo planes for travel time calculations.
4. Calculate the travel time from Gotham City to each identified filming location and supercar factory using the 
cargo plane.
5. Compile all the location data (coordinates, names, and travel times) into a DataFrame ready for plotting.
6. Create a Plotly scatter map using the compiled DataFrame, ensuring to set the color of the points based on 
travel time and to adhere to all plotting rules stated in the task.
7. Attempt to save the generated figure in both PNG and HTML formats, adhering to the stated export policy.
8. Return the figure as per the required format using `final_answer(fig)`.


```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: First, I will search for Batman filming locations worldwide, including their coordinates. I will compile  
this information into a suitable format for later use.                                                             
                                                                                                                   
<code>                                                                                                             
batman_filming_locations_search = web_agent(                                                                       
    task="Find all Batman filming locations worldwide, along with their coordinates (latitude and longitude).",    
    additional_args={}                                                                                             
)                                                                                                                  
print(batman_filming_locations_search)                                                                             
                                                                                                                   

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_filming_locations_search = web_agent(                                                                     
      task="Find all Batman filming locations worldwide, along with their coordinates (latitude and longitude).",  
      additional_args={}                                                                                           
  )                                                                                                                
  print(batman_filming_locations_search)                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

Check check_reasoning_and_plot failed with error: Make sure to save the plot under saved_map.png!

[Step 15: Duration 3.83 seconds| Input tokens: 209,858 | Output tokens: 5,871]

Reached max steps.

[Step 16: Duration 18.58 seconds| Input tokens: 235,518 | Output tokens: 7,230]

'To address the user query, I will follow these steps:\n\n1. **Gather the Required Data**: Compile a list of Batman filming locations worldwide and supercar factories, along with their coordinates.\n2. **Calculate Travel Times**: Compute the travel time to these locations from Gotham City (40.7128° N, 74.0060° W) using the cruising speed of a cargo plane.\n3. **Prepare a DataFrame**: Create a DataFrame containing names, coordinates, and travel times.\n4. **Create a Plotly Scatter Map**: Generate a map visualizing these locations based on calculated travel times, ensuring to comply with the plot creation rules.\n5. **Export the Figure**: Attempt to save the generated figure as a PNG file, followed by HTML backup export.\n\n### Step-by-step Implementation:\n\nLet\'s complete this using proper code execution.\n\n<code>\nimport numpy as np\nimport pandas as pd\nimport plotly.express as px\nimport plotly.io as pio\n\n# Coordinates of Gotham\ngotham_coords = (40.7128, -74.0060)\n\n# Batman f

I don't know how that went in your run, but in mine, the manager agent skilfully divided tasks given to the web agent in `1. Search for Batman filming locations`, then `2. Find supercar factories`, before aggregating the lists and plotting the map.

Let's see what the map looks like by inspecting it directly from the agent state:

In [14]:
fig = manager_agent.python_executor.state.get("fig")
fig if fig is not None else print("No figure found in agent state. Make sure the previous run succeeded.")

![output map](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/unit2/smolagents/output_map.png)